In [12]:
import numpy as np
import simpy
import random
import matplotlib
import pandas as pd

In [13]:
SEED = 930
UNIFORM_INTERVAL = [4.17, 5.37]
NUM_SERVER = 2
SERVER_UTILIZATIONS_PHASE1 = [0.6, 0.7, 0.8, 0.9]
SERVER_SERVICE_MEAN_PHASE1 = [5.725,6.679,7.634,8.588]
SERVER_UTILIATION_PHASE2 = 0.8
NUMBER_OF_CUSTOMERS = 100


In [14]:
phase_1_lib = {"num_server":NUM_SERVER,"arr_interval":UNIFORM_INTERVAL,"util":SERVER_UTILIZATIONS_PHASE1,"serv_mean":SERVER_SERVICE_MEAN_PHASE1}
phase_2_lib = {"num_server":NUM_SERVER,"arr_interval":UNIFORM_INTERVAL,"util":SERVER_UTILIATION_PHASE2,"serv_mean":7.634}

In [15]:
class Job(object):
    def __init__(self, name, env, operator , run_):
        self.env = env
        self.name = name
        self.operator = operator
        self.arrival_t = self.env.now
        self.action = self.env.process(self.call())
        self.run = run_
        
    
    def call(self):
        print('%s initiated at %g' % (self.name, self.env.now))
        self.run.num_jobs_c += 1
        self.run.num_jobs_system[len(self.run.num_jobs_system)-1][2] = self.env.now
        self.run.num_jobs_system.append([self.run.num_jobs_c,self.env.now,-1])
 
        with self.operator.request() as req:
            yield req
            print('%s is assigned to an operator at %g' % (self.name, self.env.now))
            self.run.queue_w_times.append(self.env.now - self.arrival_t)
            yield self.env.process(self.progress_job())
            print('%s is done at %g' % (self.name, self.env.now))
            self.departure_t = self.env.now
            self.run.num_jobs_c -= 1
            self.run.num_jobs_system[len(self.run.num_jobs_system)-1][2] = self.env.now
            self.run.num_jobs_system.append([self.run.num_jobs_c,self.env.now,-1])

            
    def progress_job(self):
        duration = random.expovariate(self.run.serv_rate)
        yield self.env.timeout(duration)
        self.run.service_times.append(duration)
        self.departure_t = self.env.now
        

In [16]:
class Run():
    def __init__(self,arr_rate,serv_rate):
        self.seed = random.random()*425
        self.run = False
        self.arr_rate = arr_rate
        self.serv_rate = 1/serv_rate
        self.num_jobs_c = 0
        self.num_jobs_system = [[0,0,-1]]
        self.jobs=[]
        self.service_times=[]
        self.queue_w_times=[]
        self.env = simpy.Environment()
        self.operator = simpy.Resource(self.env, capacity = NUM_SERVER)
    
        


    def run_sim(self):
        random.seed(self.seed)
        self.env.process(self.jobs_generator())
        self.env.run()
        self.run = True
        self.average_num_jobs_system()
        
    def show_run(self):
        if self.run!=True:
            self.run_sim()
        
    def average_num_jobs_system(self):
        weighted_value =0
        for arr in self.num_jobs_system:
            if(arr[2]==-1):
                end_of_sim = arr[1]
                break
            weighted_value += arr[0]*(arr[2]-arr[1])
        self.avg_jobs = weighted_value / end_of_sim

    def jobs_generator(self):
        for i in range(NUMBER_OF_CUSTOMERS):
            yield self.env.timeout(random.expovariate(self.arr_rate))
            job = Job('Job %s' %(i+1), self.env, self.operator,self)
            self.jobs.append(job)  

In [17]:
class Phase():
    def __init__(self,lib,num_runs):
        self.num_server = lib["num_server"]
        self.arr_int = 1/((lib["arr_interval"][0]+lib["arr_interval"][1])/2)
        self.util = lib["util"]
        self.serv_mean = lib["serv_mean"]
        self.init_runs(num_runs)
        self.num_runs = num_runs

    def init_runs(self,num_runs):
        self.runs = []
        for util_no in range(len(self.util)):
            for i in range(num_runs):
                obj = Run(self.arr_int,self.serv_mean[util_no])
                self.runs.append(obj)

    def run_phase(self):
        for run in self.runs:
            run.run_sim()
            
        print("total run :",len(self.runs)," ")
    
    def create_df(self,util):
        for i in range(len(self.ens_avg[util])):
            if(i == 0):
                ens_sum_avg = [self.ens_avg[util][0]]
                ens_cum_avg = [self.ens_avg[util][0]]
            else:
                ens_sum_avg.append((ens_sum_avg[len(ens_sum_avg)-1]+self.ens_avg[util][i])) 
                ens_cum_avg.append((ens_sum_avg[len(ens_sum_avg)-1]/(i+1)))
        df_arr_avg = pd.DataFrame(self.arr_avg[util],columns=["ARRIVAL"])
        df_ens_avg = pd.DataFrame(self.ens_avg[util],columns=["AVG"])
        df_ens_sum_avg = pd.DataFrame(ens_sum_avg,columns=["SUM-AVG"])
        df_ens_cum_avg = pd.DataFrame(ens_cum_avg,columns=["CUM-AVG"])
        df = pd.concat([df_arr_avg,df_ens_avg,df_ens_sum_avg,df_ens_cum_avg],ignore_index=True,axis=1)

        selected_runs = self.runs[(util)*self.num_runs:(util+1)*self.num_runs]
        
        for run_no, run in enumerate(selected_runs):
            df_time_spend = pd.DataFrame(np.sum([run.service_times,run.queue_w_times],axis=0),columns=["REPLICATION-"+str(run_no)])
            
            print(df_time_spend)
            df = pd.concat([df,df_time_spend],ignore_index=True,axis=1)
        
        return df

    def arrival_averages(self):
        self.arr_avg = [[0 for i in range(NUMBER_OF_CUSTOMERS)] for j in range(len(self.util))]
        for util_no in range(len(self.util)):
            for run_no in range(self.num_runs):
                for i in range(NUMBER_OF_CUSTOMERS):
                    self.arr_avg[util_no][i] = self.arr_avg[util_no][i] + self.runs[util_no*self.num_runs+run_no].jobs[i].arrival_t
        self.arr_avg = np.divide(self.arr_avg,self.num_runs)

    def ensemble_averages(self):
        self.ens_avg = [[0 for i in range(NUMBER_OF_CUSTOMERS)] for j in range(len(self.util))]
        for k in range(len(self.util)):
            for run_no in range(self.num_runs):
                for i in range(NUMBER_OF_CUSTOMERS):
                    self.ens_avg[k][i] = self.ens_avg[k][i] + (self.runs[k*self.num_runs+run_no].jobs[i].departure_t - self.runs[k*self.num_runs+run_no].jobs[i].arrival_t)
        self.ens_avg = np.divide(self.ens_avg,self.num_runs)
            

In [18]:
phase_1 = Phase(phase_1_lib,10)
phase_1_30 = Phase(phase_1_lib,30)

In [19]:
phase_1.run_phase()
phase_1_30.run_phase()

Job 1 initiated at 0.424013
Job 1 is assigned to an operator at 0.424013
Job 2 initiated at 2.92627
Job 2 is assigned to an operator at 2.92627
Job 2 is done at 6.08751
Job 1 is done at 6.64599
Job 3 initiated at 11.2527
Job 3 is assigned to an operator at 11.2527
Job 4 initiated at 12.9242
Job 4 is assigned to an operator at 12.9242
Job 4 is done at 13.0798
Job 3 is done at 16.3883
Job 5 initiated at 18.5384
Job 5 is assigned to an operator at 18.5384
Job 5 is done at 20.6544
Job 6 initiated at 26.911
Job 6 is assigned to an operator at 26.911
Job 6 is done at 27.6642
Job 7 initiated at 31.0007
Job 7 is assigned to an operator at 31.0007
Job 7 is done at 37.3926
Job 8 initiated at 40.9497
Job 8 is assigned to an operator at 40.9497
Job 8 is done at 42.6392
Job 9 initiated at 46.3198
Job 9 is assigned to an operator at 46.3198
Job 10 initiated at 47.4643
Job 10 is assigned to an operator at 47.4643
Job 11 initiated at 47.6619
Job 9 is done at 56.428
Job 11 is assigned to an operator at

In [20]:
phase_1.ensemble_averages()
phase_1.arrival_averages()

phase_1_30.ensemble_averages()
phase_1_30.arrival_averages()

In [21]:
phase_1.ensemble_averages
phase_1_30.ensemble_averages

<bound method Phase.ensemble_averages of <__main__.Phase object at 0x127e65ad0>>

In [22]:
df_0 = phase_1.create_df(0)
df_1 = phase_1.create_df(1)
df_2 = phase_1.create_df(2)
df_3 = phase_1.create_df(3)
df_0.to_csv("df_0.6_10.csv")
df_1.to_csv("df_0.7_10.csv")
df_2.to_csv("df_0.8_10.csv")
df_3.to_csv("df_0.9_10.csv")

df_0_30 = phase_1_30.create_df(0)
df_1_30 = phase_1_30.create_df(1)
df_2_30 = phase_1_30.create_df(2)
df_3_30 = phase_1_30.create_df(3)
df_0_30.to_csv("df_0.6_30.csv")
df_1_30.to_csv("df_0.7_30.csv")
df_2_30.to_csv("df_0.8_30.csv")
df_3_30.to_csv("df_0.9_30.csv")

    REPLICATION-0
0        3.161233
1        6.221975
2        0.155628
3        5.135570
4        2.115912
..            ...
95       9.087870
96      13.293029
97       1.074530
98       1.927855
99       7.426067

[100 rows x 1 columns]
    REPLICATION-1
0        4.843836
1        8.201928
2       11.304855
3        8.983343
4        9.935738
..            ...
95      16.213912
96      16.840618
97      21.294099
98      17.296450
99      21.947813

[100 rows x 1 columns]
    REPLICATION-2
0        3.506524
1        1.971140
2        6.112700
3       19.045124
4        3.554304
..            ...
95       1.057546
96       2.199318
97       1.459071
98       2.740724
99      28.114392

[100 rows x 1 columns]
    REPLICATION-3
0        0.230762
1        2.105197
2        0.831076
3        4.255254
4        3.024389
..            ...
95       7.514583
96       1.065478
97       0.997674
98       0.554343
99      11.317657

[100 rows x 1 columns]
    REPLICATION-4
0        0.093021
1   